In [22]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as firefoxOptions
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### получение погоды
**Рязань**(city_code=4394)

In [2]:
col = ['year', 'month', 'day', 'td', 'pd', 'cloudd', 'precipd', 'windd', 'tn', 'pn', 'cloudn', 'precipn', 'windn']

In [3]:
def get_year_weather(year, mon_st=1, mon_fin=12, city_code=4394):
    
    url = "http://www.gismeteo.ru/diary"
    weather = []
    month_start = mon_st
    while month_start <= mon_fin:
        options = firefoxOptions()
        options.add_argument('--headless')
        options.add_argument("--no-sandbox")
        
        browser = webdriver.Firefox(options=options)
        browser.get(f"{url}/{city_code}/{year}/{month_start}/")
        
        soup = BeautifulSoup(browser.page_source, 'html5lib')
        table = soup.find('table')
                
        day = [year, month_start]
        for i in table.find_all('td'):
            if len(day) == 13:
                weather.append(day)
                day = [year, month_start]
            if len(i.text) == 0 and i.find('img', class_='label_icon label_small screen_icon') is not None:
                day.append(str(i.find('img', class_='label_icon label_small screen_icon'))[88:].strip('png"/>').strip('.'))
            elif len(i.text) == 0 and i.find('img', class_='label_icon label_small screen_icon') is None:
                day.append(np.nan)
            if len(i.text) != 0:
                day.append(i.text)
        weather.append(day)
        
        month_start += 1
        browser.quit()
    weather = pd.DataFrame(weather, columns=col)
    weather.to_csv(f'{year}_{mon_st}_{mon_fin}.csv')
    print(f'{year} done!')

In [7]:
# get_year_weather(2023, mon_st=1, mon_fin=4); #

##### и корректировка полученных данных с сохранением в new_data.csv

In [58]:
# new_data = pd.read_csv('E:\\Jupyter\\pogoda\\NEW_weather\\1997.csv', index_col=0)
# weather = pd.read_csv('E:\Jupyter\pogoda\study-of-the-Ryazan-weather\weather.csv')
# years = weather['year'].unique()
# for i in years[1:]:
#      new_data = pd.concat([new_data, pd.read_csv(f'E:\\Jupyter\\pogoda\\NEW_weather\\{i}.csv', index_col=0)], axis=0)  

In [59]:
# new_data.insert(loc=7, column='wdird', value=new_data['windd'].apply(lambda x: x.split()[0]))
# new_data.insert(loc=8, column='wsd', value=new_data['windd'].apply(lambda x: x.split()[1].strip('м/с') if len(x.split())==2 else 0))
# new_data.insert(loc=15, column='wdirn', value=new_data['windn'].apply(lambda x: x.split()[0]))
# new_data.insert(loc=16, column='wsn', value=new_data['windn'].apply(lambda x: x.split()[1].strip('м/с') if len(x.split())==2 else 0))
# new_data = new_data.replace('still.gif', np.NAN)
# new_data['td'] = new_data['td'].apply(lambda x: str(x).strip('+')).astype({'td': 'float'})
# new_data['tn'] = new_data['tn'].apply(lambda x: str(x).strip('+')).astype({'tn': 'float'})

In [60]:
# new_data['wsd'] = np.where(new_data['pd'].isna(), np.NAN, new_data['wsd'])
# new_data['wsn'] = np.where(new_data['tn'].isna(), np.NAN, new_data['wsn'])
# new_data['cloudd'] = np.where(new_data['cloudd']=='—', np.NAN, new_data['cloudd'])
# new_data['cloudn'] = np.where(new_data['cloudn']=='—', np.NAN, new_data['cloudn'])
# new_data.drop(labels=['windd','windn'], axis=1, inplace=True)

In [54]:
new_data.reset_index(drop=True).to_csv('new_data.csv')

In [55]:
new_data = pd.read_csv('new_data.csv', index_col=0)

In [61]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8943 entries, 0 to 8942
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     8943 non-null   int64  
 1   month    8943 non-null   int64  
 2   day      8943 non-null   int64  
 3   td       8928 non-null   float64
 4   pd       8928 non-null   float64
 5   cloudd   8919 non-null   object 
 6   precipd  1352 non-null   object 
 7   wdird    8928 non-null   object 
 8   wsd      8928 non-null   float64
 9   tn       8915 non-null   float64
 10  pn       8915 non-null   float64
 11  cloudn   8904 non-null   object 
 12  precipn  1201 non-null   object 
 13  wdirn    8915 non-null   object 
 14  wsn      8915 non-null   float64
dtypes: float64(6), int64(3), object(6)
memory usage: 1.1+ MB
